In [ ]:
%load_ext autoreload

# 写在前面：TMDAT数据格式介绍

tmdat数据结构如下，分三个文件夹等级

1. dataset，里面涵盖多个样本
2. 视频样本，命名为作为读取数据集时的匹配关键字matchkey
3. cone和rod文件夹，分别存储多通路数据

```
├── dataset
│   ├── matchkey
│   │   ├── cone
│   │       ├── info.txt
│   │       ├── xxx.tmdat
│   │   ├── rod
│   │       ├── info.txt
│   │       ├── xxx.tmdat
```


# (1) 仅读取CONE（RGB）数据

### 功能
- 给定所录制的数据的cone文件夹下的tmdat格式文件路径，返回RGB或者RAW格式数据，存储为npy
- 调用 tianmoucv.data.cone_tmdat_to_npy

In [ ]:
%autoreload
import numpy as np
import matplotlib.pyplot as plt
from tianmoucv.data import cone_tmdat_to_npy,rod_tmdat_to_npy

cone_data_path = '/data/shared_dataset/test_metacam_encode/cone/cone_encoded.tmdat'

cone_RGB, cone_time_stampe = cone_tmdat_to_npy(cone_data_path,idx=2,mode='RGB')
plt.figure(figsize=(3,6))
plt.imshow(cone_RGB)
plt.show()

print('时间戳:',cone_time_stampe,' *10 us')

# (2) 仅读取ROD（TD,SDL,SDR）数据

### 功能
- 给定所录制的数据的rod文件夹下的tmdat格式文件路径，返回三通道数据，存储为npy
- 调用 tianmoucv.data.cone_tmdat_to_npy
- 第一个通道为TD，而后为SD-left和SD-right

In [ ]:
%autoreload
import numpy as np
import matplotlib.pyplot as plt
from tianmoucv.data import rod_tmdat_to_npy,multiple_rods_tmdat_to_npy

import time

rod_data_path = '/data/shared_dataset/test_metacam_encode/rod/rod_encoded.tmdat'

rod_RGB,rod_time_stampe = rod_tmdat_to_npy(rod_data_path, idx=26)
print(rod_RGB.shape)
rod_RGB /= 128
plt.figure(figsize=(2,2))
plt.imshow(rod_RGB.transpose(1,2,0))
plt.show()

print('时间戳:',rod_time_stampe,' *10 us')

# 一次性连续读取N帧
rod_data_path = '/data/shared_dataset/test_metacam_encode/rod/rod_encoded.tmdat'
rod_RGBs,rod_time_stampe = multiple_rods_tmdat_to_npy(rod_data_path, idx=25,length=26)
print(rod_RGBs.shape)
rod_RGB_t = rod_RGBs[:,1,...]
rod_RGB_t /= 128
plt.figure(figsize=(2,2))
plt.imshow(rod_RGB.transpose(1,2,0))
plt.show()

print('时间戳:',rod_time_stampe,' *10 us')
print('差异：',np.sum(rod_RGB_t-rod_RGB))

t1= 0
t2= 0
for i in range(26*100):
    times=time.time()
    rod_RGB,rod_time_stampe = rod_tmdat_to_npy(rod_data_path, idx=50,print_info=False)
    timee=time.time()
    t1 += timee-times

for i in range(100):
    times=time.time()
    rod_RGB,rod_time_stampe = multiple_rods_tmdat_to_npy(rod_data_path, idx=50,length=26,print_info=False)
    timee=time.time()
    t2+= timee-times

print('test_speed:',t2,t1)

# (3) 以数据集形式同步读取AOP和COP，默认做对齐和ISP处理

- TianmoucDataReader
    - ## [输入]
    - 输入dataPath：包含1个或多个子目录，每个子目录对应1段Tianmouc数据。
        - 支持string格式(仅输入1个地址)或list格式(输入1个或多个地址)。
           - 自动扫描这个(些)地址下所有的tmdat数据，并以文件夹名区分
           - 用matchkey读取特定sample
           - 不指定matchkey时混合读取所有数据（例如用于神经网络训练）
        - 对于单目数据，每个sample下应包含rod和cone两个目录，多目数据额外还有目录rod_N和cone_N，N为相机编号N>=1
          
    - 输入N：返回的dataset中包含多个sample，每个sample包含(N+1)帧COP，以及中间的所有AOP帧。
        - 默认N=1，在757fps模式下sample中有F0，F1两帧COP，以及中间的(25+1)帧AOP，最后一帧AOP与下一个sample第1帧AOP相同，可以跳过。
          
    - 输入camera_idx：默认为0，表示识别单目输入，若为双目数据，则camera_idx=0,1分别录取双目数据。
    - 参数MAXLEN强制默认设为-1，即始终为扫描全部数据，否则对每个数据长度做截断。
      
    - ## [输出]
    - 通过sample = dataset[index]逐一获取数据
    - sample为字典类型，包含如下数据，以对应字符串作为key进行索引使用
        - COP
            - COP的精确帧率为30.3fps
            - 'F0'使用ISP处理
            - 'F0_without_isp'不加额外处理，若加红外滤光片应使用这个数据
            - 'F0_HDR'为简易融合算法处理结果，由同步的SD和RGB合成高动态图
        - AOP
            - 'rawDiff'为AOP像素原始输出(160×160), 为tianmuocv非神经网络预处理接口的输入
            - 'tsdiff'为rawDiff直接插值得到的与COP同分辨率的图像(320×640), 用于神经网络的输入
            - 上述三个对应的key_value均为张量格式，torch.Size([3, X, height, width])
                - 第0个维度为3，分别依次对应TD，SD1，SD2
                - 第1个维度对应AOP帧数目，在757fps模式下X=N×25+1, 每25帧为一个单位
                - 第2，3个维度对应AOP帧的分辨率
        - 'sysTimeStamp'为系统初始时间，用于在多目相机情况下进行时间对齐。
            - 两相机之间初始时间差为sysTimeStamp1-sysTimeStamp2，单位为秒
        - 'labels'用于标注HDR，HS，Blur，Noisy等4种极端情况分类，暂未实装。
        - 'meta'包含了该段目录的一些元数据，如文件存储目录，时间戳等等，需要详细数据分析时使用
        

In [ ]:
%autoreload
import sys,os
import numpy as np
import torch
import matplotlib.pyplot as plt
from tianmoucv.isp import SD2XY
from tianmoucv.data import TianmoucDataReader
import torch.nn.functional as F
import cv2

train='/data/lyh/tianmoucData/tianmoucReconDataset/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]
val='/data/lyh/tianmoucData/tianmoucReconDataset/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]
key_list = []
print('---------------------------------------------------')
for sampleset in traindata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
print('---------------------------------------------------')
for sampleset in valdata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
        
all_data = valdata + traindata

## （4）检查数据完整性

- 如果数据有严重丢帧，又想要保证读出大多数数据，可以设置strict=False
- 为了加速整个数据集读取，可以使用cachePath，详见源码

In [ ]:
%autoreload
from tianmoucv.data import TianmoucDataReader
dataset = TianmoucDataReader("/data/shared_dataset/test_metacam_encode/",strict=True)
print(dataset.get_clip_completeness())
print(dataset.__len__())

## （5）读取数据与可视化示范

使用tianmoucv自带的差分数据可视化接口，可以获得标准的可视化色彩

- 接口: tianmoucv.isp import vizDiff
- vizDiff(diff,thresh=0,bg_color='white',gain=1)
- diff： TD/SDL/SDR
- thresh：可视化阈值
- bg_color：white/black/gray gray输出1通道灰度
- gain：数字增益
- 输入输出均为torch.Tensor

In [ ]:
%autoreload
from tianmoucv.isp import vizDiff


aim = 4 # 读取的目标COP帧的帧号

# 这些形式都可以读取数据
datapath = "/data/lyh/tianmoucData/tianmoucReconDataset/train/normal/train_cross2/"
datapath = [train,val]
datapath = valdata + traindata
key_list = ['train_cross2']
color_modes =  ['white','black','gray']

_correct_sd_upsampler = True

for key in key_list:
    dataset = TianmoucDataReader(datapath,
                                 matchkey=key,
                                 camera_idx=0,
                                 _correct_sd_upsampler = _correct_sd_upsampler)
    img_list = []
    for index in range(aim,min(aim+1,len(dataset))):
        sample = dataset[index]

        if _correct_sd_upsampler:
            txydiff = torch.Tensor(sample['txydiff'])
        else:
            txydiff = torch.Tensor(sample['tsdiff'])
        length = txydiff.shape[1]
        dataRatio = sample['dataRatio']
        print('这个sample中，RGB(COP)和AOP的帧率比率是:',dataRatio)
        plt.figure(figsize=(12,2))  
        
        F = sample['F0'].numpy() #第i帧COP
        F_HDR = sample['F0_HDR'].numpy() #第i帧的HDR合成结果
        F_HDR[F_HDR>1]=1
        F_HDR[F_HDR<0]=0

        txydiff_i = txydiff[:,0:dataRatio,...] #第i帧到第i+1帧COP之间的AOP数据
        plt.subplot(1,3,1)
        plt.imshow(F)
        plt.subplot(1,3,2)
        plt.imshow(txydiff_i[:,0,...].permute(1,2,0)*16) #可视化第0帧，乘个Gain方便看
        plt.subplot(1,3,3)
        plt.imshow(F_HDR)
        plt.show()
            
        for j in range(3):
            td = txydiff_i[0,0,...]
            Ix = txydiff_i[1,0,...]
            Iy = txydiff_i[2,0,...]
            td_viz = vizDiff(td,thresh=2,bg_color=color_modes[j],gain=2) /256.0
            Ix_viz = vizDiff(Ix,thresh=2,bg_color=color_modes[j],gain=2) /256.0
            Iy_viz = vizDiff(Iy,thresh=2,bg_color=color_modes[j],gain=2) /256.0
            plt.subplot(3,3,1+j*3)
            plt.imshow(td_viz)
            plt.subplot(3,3,2+j*3)
            plt.imshow(Ix_viz)
            plt.subplot(3,3,3+j*3)
            plt.imshow(Iy_viz)   
        plt.show()
    break

# (6)读取多目数据
- 可以使用系统时间戳对齐
- 注意设置相机序号，从0开始

In [ ]:
%autoreload
import sys,os
import numpy as np
import torch
import torch.nn.functional as F
import cv2
import math,time
import matplotlib.pyplot as plt

from tianmoucv.isp import SD2XY

aim = 4
N = 3   # read continue N frames

dataset_top = '/data/yuguo/dataset/checkerboard_20240202'
key='test10'
dataset_0 = TianmoucDataReader(dataset_top,N=N,matchkey=key,camera_idx=0,_correct_sd_upsampler=True) #index0: rod
dataset_1 = TianmoucDataReader(dataset_top,N=N,matchkey=key,camera_idx=1,_correct_sd_upsampler=True) #index1: rod_1

#随便读一个sample，把里面的sysTimeStamp拿出来
sample_l = dataset_0[0]
sample_r = dataset_1[0]
ts1 = sample_l['sysTimeStamp']
ts2 = sample_r['sysTimeStamp']
index_bias = (ts2-ts1)/1000.0/33
print(ts1,ts2,'两相机的时间差:',(ts2-ts1)/1000.0,'ms,对应的index 之差为:',index_bias)
#然后怎么处理都行

img_list = []
for index in range(aim,min(aim+1,len(dataset))):
    sample_l = dataset_0[index]
    sample_r = dataset_1[index]
    
    tsdiff = torch.Tensor(sample_l['txydiff'])
    length = tsdiff.shape[1]
    print('一次性读出N:',N,'个RGB帧以及与其同步的AOP')
    dataRatio = sample['dataRatio']
    print('这个sample中，RGB(COP)和AOP的帧率比率是:',dataRatio)
    plt.figure(figsize=(12,2*N))  
    for i in range(N):
        F = sample_l['F'+str(i)].numpy()
        F_HDR = sample_l['F'+str(i)+'_HDR'].numpy()
        F_HDR[F_HDR>1]=1
        F_HDR[F_HDR<0]=0
        plt.subplot(N,6,1+i*6)
        plt.imshow(F)
        plt.subplot(N,6,2+i*6)
        plt.imshow(tsdiff[:,i*dataRatio,...].permute(1,2,0)*16)
        plt.subplot(N,6,3+i*6)
        plt.imshow(F_HDR)

    tsdiff = torch.Tensor(sample_r['txydiff'])
    length = tsdiff.shape[1]
    print('一次性读出N:',N,'个RGB帧以及与其同步的AOP')
    dataRatio = sample['dataRatio']
    print('这个sample中，RGB(COP)和AOP的帧率比率是:',dataRatio)
    for i in range(N):
        F = sample_r['F'+str(i)].numpy()
        F_HDR = sample_r['F'+str(i)+'_HDR'].numpy()
        F_HDR[F_HDR>1]=1
        F_HDR[F_HDR<0]=0
        plt.subplot(N,6,4+i*6)
        plt.imshow(F)
        plt.subplot(N,6,5+i*6)
        plt.imshow(tsdiff[:,i*dataRatio,...].permute(1,2,0)*16)
        plt.subplot(N,6,6+i*6)
        plt.imshow(F_HDR)
        
    plt.show()
